In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from cryotherm.material_db import MaterialDatabase
from cryotherm.stage import Stage
from cryotherm.conduction import Conduction
from cryotherm.radiation import Radiation
from cryotherm.solver import ThermalModel

from cryotherm import DATA_PATH

db = MaterialDatabase(DATA_PATH)

def fridge_curve(T):
    """ Sample Curve for the Cryotel GT Fridge"""
    # load the fridge curve data from the csv file
    fridge_temp, fridge_lift = np.loadtxt(os.path.join(DATA_PATH, "fridges", "CryotelGT_LoadCurve.csv"), unpack=True)
    

    # interpolate the fridge curve
    fridge_interp = np.interp(T, fridge_temp, fridge_lift)

    # if the temperature is below the minimum, return 0
    fridge_interp[T < fridge_temp.min()] = 0
    return fridge_interp

# plot the fridge curve

T = np.linspace(30, 100, 100)
plt.figure(figsize=(8, 5))
plt.plot(T, fridge_curve(T), label="Fridge Curve")
plt.xlabel("Temperature (K)")
plt.ylabel("Load (W)")
plt.title("Fridge Curve for Inner Stage")
plt.grid()
plt.legend()
plt.show()

# stages
outer  = Stage("Outer", 300, fixed=True)
middle = Stage("Mid", 150, external_load=0.3)
inner  = Stage("Inner", 80, fixed = True)



# links
strap1 = Conduction(outer, middle, length=0.05, area=1e-5,
                    material="Al6061", mat_db=db)
strap2 = Conduction(middle, inner, length=0.02, area=5e-5,
                    material="Al6061", mat_db=db)

rad1 = Radiation(outer, middle, emissivity=0.1, area=0.01)
rad2 = Radiation(middle, inner, emissivity=0.2, area=0.005)

mdl = ThermalModel([outer, middle, inner],
                   conductors=[strap1, strap2],
                   radiators=[rad1, rad2])
mdl.solve()

for s in mdl.stages:
    print(s)


ValueError: could not convert string '35.80619720284622,' to float64 at row 0, column 1.

<Figure size 800x500 with 0 Axes>